In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import plot_importance

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
df = pd.read_csv('data/Android_Malware.csv', low_memory = False)
df

### Dzielenie na train/test/ walidacje oraz analiza danych

In [ ]:
X_train, X_test_tmp, y_train, y_test_tmp = train_test_split(
    df[df.columns.values[:-1]],
    df[df.columns.values[-1]],
    test_size=0.4, random_state=213)

In [ ]:
X_test, X_validation, y_test, y_validation = train_test_split(
    X_test_tmp,
    y_test_tmp,
    test_size=0.5, random_state=7)

In [ ]:
df = X_train 
df["Label"] = y_train

In [ ]:
df.info()

jak wygladaja najpopularniejsze wartosci w kazdej z kolumn

In [ ]:
for col in df.columns:
    print(f"Top 10 most common values in {col}:")
    print(df[col].value_counts().nlargest(10))
    print()

ilosc unikalnych wartosci w kazdej z kolumn

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df.nunique(dropna=False))

rozklad przewidywanej cechy

In [ ]:
df.iloc[:, -1].hist()

"Adware jest niechcianym oprogramowaniem służącym do wyświetlania reklam na Twoim ekranie". Adware generuje przychody dla swoich twórców dzięki automatycznemu wyświetlaniu internetowych reklam w interfejsie użytkownika danej aplikacji lub w formie okienek wyskakujących podczas procesu instalacji.

Scareware is a type of malware attack that claims to have detected a virus or other issue on a device and directs the user to download or buy malicious software to resolve the problem. Generally speaking, scareware is the gateway to a more intricate cyberattack and not an attack in and of itself.

benign - having no harmful influence or effect

Wszystkie NA są skumulowane we wierszach które można policzyć na palcach jedner ręki można je usunąć

In [ ]:
df = df.dropna()

Usunięcie kolumn które mają tylko jedną wartość

In [ ]:
col_to_drop = df.columns[df.nunique() <= 1]
df = df.drop(columns = col_to_drop)

Usunięcie kolumn ID, i dwóch kolumn w których są same zera ale zapisane na różne sposoby (int, str, float)

Ta kolumna ma wartości które są int i float, zrzutujemy je na int

In [ ]:
df.info()

### Usuwanie skorelowanych kolumn oraz transformacja danych

In [ ]:
def select_correlated_columns_to_remove(df, corr_treshold):
    cor_lis = []
    cor = df.corr(method="spearman")
    for i, _ in enumerate(cor.columns):
        for j, col_name_to_drop in enumerate(cor.columns):
            if i < j and (abs(cor.iloc[i,j]) > corr_treshold or math.isnan(cor.iloc[i,j])):
                cor_lis.append(col_name_to_drop)
    return cor_lis

Użyliśmy na początku korelacji spearmana, żeby zostawić jak najmniej kolumn i sprawdzić, które z nich są wartościowe. Po policzeniu Auc odrzuciliśmy prawie wszystkie oryginalne kolumny. Zatem nie wracaliśmy już do tego punktu.

In [ ]:
cor_lis = select_correlated_columns_to_remove(df, 0.95)
df=df.drop(cor_lis,axis =1 )

In [ ]:
def transform_data(X,y):
    df = X
    df["Label"] = y
    df = df.dropna()
    df = df.drop(columns = col_to_drop)
    df = df.drop(columns = ["Unnamed: 0", "Flow ID", " CWE Flag Count", "Fwd Avg Bytes/Bulk",' Protocol',' Down/Up Ratio'])
    
    ip = list(df[' Source IP'])
    ip = [[int(ip[i].split('.')[j]) for j in range(len(ip[i].split('.')))] for i in range(len(ip))]
    ip = pd.DataFrame(ip).fillna(0)
    ip.columns = ['Source IP1','Source IP2',"Source IP3","Source IP4"]
    ip2 = list(df[' Destination IP'])
    ip2 = [[int(ip2[i].split('.')[j]) for j in range(len(ip2[i].split('.')))] for i in range(len(ip2))]
    ip2 = pd.DataFrame(ip2).fillna(0)
    ip2.columns = ['Destination IP1','Destination IP2',"Destination IP3","Destination IP4"]
    ip = pd.concat([ip,ip2], axis = 1)
    df = pd.concat([df.reset_index(),ip],axis = 1).drop('index', axis = 1)
    
    mapping_dict = {"Benign": 0,
                "Android_Scareware":1,
                "Android_Adware": 2,
                "Android_SMS_Malware":3}
    
    df["Label"] = pd.Series(df["Label"]).map(mapping_dict)
 
    df = df.drop(columns = cor_lis,errors='ignore')
    return df.drop("Label", axis=1), df["Label"]

In [ ]:
X_test, y_test = transform_data(X_test, y_test)
X_train, y_train = transform_data(X_train, y_train)

### Wybór kolumn po tranformacji danych

In [ ]:
""""
Sposób w jaki liczyliśmy auc

from feature_engine.selection import SelectBySingleFeaturePerformance

sel1 = SelectBySingleFeaturePerformance(
    estimator=RandomForestClassifier(random_state=1),
    scoring='roc_auc'
)

for i in range(4):
    Y = np.select([y_train == i], [1], 0 )
    sel1.fit(X_train, Y)
    print(sel1.feature_performance_)
"""


Nie ma sensu za każdym razem liczyć auc, więc wczytamy już policzone. 
Każde auc odpowiada za wykrywanie 1 kategorii więc jest ich łącznie 4.

In [ ]:
auc0={' Source Port': 0.6667803137571054,
 ' Destination Port': 0.5280530163444412,
 ' Flow Duration': 0.5432468532109304,
 ' Total Fwd Packets': 0.5115232835638613,
 ' Total Backward Packets': 0.5257881750154091,
 'Total Length of Fwd Packets': 0.564206596036872,
 ' Total Length of Bwd Packets': 0.5609472971943124,
 ' Fwd Packet Length Min': 0.5288376538496439,
 ' Fwd Packet Length Std': 0.5410624009741203,
 ' Bwd Packet Length Min': 0.5310371093541425,
 ' Bwd Packet Length Std': 0.5376187852377542,
 'Flow Bytes/s': 0.5226595986307652,
 ' Flow Packets/s': 0.5341000756469306,
 ' Flow IAT Std': 0.5098433977150534,
 ' Flow IAT Min': 0.5419213925068221,
 'Fwd IAT Total': 0.5396089111679344,
 ' Fwd IAT Std': 0.5107185905594186,
 ' Fwd IAT Min': 0.549299241535541,
 'Bwd IAT Total': 0.5101626024184551,
 ' Bwd IAT Std': 0.5073457412803558,
 'Fwd PSH Flags': 0.4995634750962859,
 ' Fwd Header Length': 0.5293550369946379,
 ' Bwd Packets/s': 0.523087600852063,
 'FIN Flag Count': 0.49969180699706683,
 ' PSH Flag Count': 0.5056831548783184,
 ' ACK Flag Count': 0.5075828796322425,
 ' URG Flag Count': 0.5025464541539769,
 'Init_Win_bytes_forward': 0.5686777003883696,
 ' Init_Win_bytes_backward': 0.5744043850502205,
 ' act_data_pkt_fwd': 0.5155759304794502,
 ' min_seg_size_forward': 0.5082642067158343,
 'Active Mean': 0.5039074595954259,
 ' Active Std': 0.5008584278213167,
 ' Idle Std': 0.5016072510651556,
 'Source IP1': 0.5208522632633458,
 'Source IP2': 0.5224823089204553,
 'Source IP3': 0.522061468243979,
 'Source IP4': 0.5704274681398592,
 'Destination IP1': 0.5834675438830063,
 'Destination IP2': 0.6053260099365425,
 'Destination IP3': 0.6391564449566961,
 'Destination IP4': 0.6331360322937867}

auc1={' Source Port': 0.6095152607547952,
 ' Destination Port': 0.5260450208036361,
 ' Flow Duration': 0.5281273614448136,
 ' Total Fwd Packets': 0.5108599238826574,
 ' Total Backward Packets': 0.5179758046764285,
 'Total Length of Fwd Packets': 0.545840953226306,
 ' Total Length of Bwd Packets': 0.559587460354607,
 ' Fwd Packet Length Min': 0.5150061953828514,
 ' Fwd Packet Length Std': 0.5330914556927385,
 ' Bwd Packet Length Min': 0.5222729790837767,
 ' Bwd Packet Length Std': 0.5393663579909972,
 'Flow Bytes/s': 0.5141840016015934,
 ' Flow Packets/s': 0.5234415339130424,
 ' Flow IAT Std': 0.5073244426784005,
 ' Flow IAT Min': 0.5254465601232302,
 'Fwd IAT Total': 0.5254558091371572,
 ' Fwd IAT Std': 0.5068278697367655,
 ' Fwd IAT Min': 0.5280507427475585,
 'Bwd IAT Total': 0.5039745923406389,
 ' Bwd IAT Std': 0.5042883451291047,
 'Fwd PSH Flags': 0.5010867151639863,
 ' Fwd Header Length': 0.5231815422844631,
 ' Bwd Packets/s': 0.5163463778158034,
 'FIN Flag Count': 0.5012267692229343,
 ' PSH Flag Count': 0.501299177193454,
 ' ACK Flag Count': 0.5003763616250868,
 ' URG Flag Count': 0.5057836854464591,
 'Init_Win_bytes_forward': 0.5445225899283014,
 ' Init_Win_bytes_backward': 0.5488982957554105,
 ' act_data_pkt_fwd': 0.5131342031079205,
 ' min_seg_size_forward': 0.5080447640893865,
 'Active Mean': 0.4997263815266238,
 ' Active Std': 0.5000967069366542,
 ' Idle Std': 0.50039666411773,
 'Source IP1': 0.5120834867732361,
 'Source IP2': 0.513812621724962,
 'Source IP3': 0.5176831109568006,
 'Source IP4': 0.590142209910279,
 'Destination IP1': 0.5669543381574059,
 'Destination IP2': 0.5847780097124761,
 'Destination IP3': 0.61243985785373,
 'Destination IP4': 0.6058817694807083}

auc2={' Source Port': 0.619494360969236,

 ' Destination Port': 0.5522096964193727,
 ' Flow Duration': 0.565340184513613,
 ' Total Fwd Packets': 0.5288163459577616,
 ' Total Backward Packets': 0.5432622240389144,
 'Total Length of Fwd Packets': 0.5797146557503599,
 ' Total Length of Bwd Packets': 0.589504149307628,
 ' Fwd Packet Length Min': 0.5281609705719504,
 ' Fwd Packet Length Std': 0.5622176644585165,
 ' Bwd Packet Length Min': 0.5359076230719914,
 ' Bwd Packet Length Std': 0.5621001539754732,
 'Flow Bytes/s': 0.5384421326083486,
 ' Flow Packets/s': 0.5587014714464367,
 ' Flow IAT Std': 0.5355029736402083,
 ' Flow IAT Min': 0.5572710324588441,
 'Fwd IAT Total': 0.5552154921669692,
 ' Fwd IAT Std': 0.525457563232787,
 ' Fwd IAT Min': 0.557227927917605,
 'Bwd IAT Total': 0.5274787185312164,
 ' Bwd IAT Std': 0.5199107052302893,
 'Fwd PSH Flags': 0.5110209992354248,
 ' Fwd Header Length': 0.5649256821571141,
 ' Bwd Packets/s': 0.548567384472877,
 'FIN Flag Count': 0.4996152994949028,
 ' PSH Flag Count': 0.5064995958124713,
 ' ACK Flag Count': 0.5097289292362083,
 ' URG Flag Count': 0.5069323779637565,
 'Init_Win_bytes_forward': 0.5734484114927324,
 ' Init_Win_bytes_backward': 0.5742372376657341,
 ' act_data_pkt_fwd': 0.5250498204674048,
 ' min_seg_size_forward': 0.5428487384051487,
 'Active Mean': 0.5085359523478155,
 ' Active Std': 0.5018885551873704,
 ' Idle Std': 0.5026271133955946,
 'Source IP1': 0.5093481226733437,
 'Source IP2': 0.5127355525526736,
 'Source IP3': 0.5197810657086236,
 'Source IP4': 0.6292940726601796,
 'Destination IP1': 0.6000708569101265,
 'Destination IP2': 0.6179593909811868,
 'Destination IP3': 0.6542533934055511,
 'Destination IP4': 0.6413952745660961}

auc3={' Source Port': 0.6379991311189014,
 ' Destination Port': 0.5563665831240362,
 ' Flow Duration': 0.5529182319576855,
 ' Total Fwd Packets': 0.5345564020394712,
 ' Total Backward Packets': 0.5387853249046745,
 'Total Length of Fwd Packets': 0.5813068547666688,
 ' Total Length of Bwd Packets': 0.5878820742615405,
 ' Fwd Packet Length Min': 0.5340162772250147,
 ' Fwd Packet Length Std': 0.5628073596857487,
 ' Bwd Packet Length Min': 0.5443412553308494,
 ' Bwd Packet Length Std': 0.5596898660930836,
 'Flow Bytes/s': 0.5295650431784452,
 ' Flow Packets/s': 0.5470448477668078,
 ' Flow IAT Std': 0.5176486127553078,
 ' Flow IAT Min': 0.5521831300555938,
 'Fwd IAT Total': 0.5376555620659789,
 ' Fwd IAT Std': 0.512648110710675,
 ' Fwd IAT Min': 0.5392868522482441,
 'Bwd IAT Total': 0.5126170390994146,
 ' Bwd IAT Std': 0.5101814178361082,
 'Fwd PSH Flags': 0.5156249122280595,
 ' Fwd Header Length': 0.5775761891744442,
 ' Bwd Packets/s': 0.5442219795089852,
 'FIN Flag Count': 0.5018783188361402,
 ' PSH Flag Count': 0.5106739226613008,
 ' ACK Flag Count': 0.5116913166044152,
 ' URG Flag Count': 0.5036676099161546,
 'Init_Win_bytes_forward': 0.5839914725418078,
 ' Init_Win_bytes_backward': 0.5876287945470929,
 ' act_data_pkt_fwd': 0.5315391349473729,
 ' min_seg_size_forward': 0.5618074944179611,
 'Active Mean': 0.5017283625995507,
 ' Active Std': 0.5003691000455467,
 ' Idle Std': 0.5001430264872218,
 'Source IP1': 0.5192414158227315,
 'Source IP2': 0.5210207880736546,
 'Source IP3': 0.526187117293753,
 'Source IP4': 0.6431843405311595,
 'Destination IP1': 0.6118328221676916,
 'Destination IP2': 0.6287977678219656,
 'Destination IP3': 0.650626178105102,
 'Destination IP4': 0.6428527493983265}

Gdy policzymy sumę auc widzimy, że połowa zmiennych nie jest w stanie wykryć żadnej kategorii. 
Wykres przedstawia posortowane zmienne i ich średnie gini. 

In [ ]:
f = [2*(a0+a1+a2+a3)/4 -1 for a0,a1,a2,a3 in zip(list(auc0.values()),list(auc1.values()),list(auc2.values()),list(auc3.values())) ]
fcol = list(pd.DataFrame( f,list(auc1.keys())).sort_values(by = 0, ascending = False).index)
auc = pd.DataFrame( f,list(auc1.keys())).sort_values(by = 0, ascending = False).reset_index()
sns.barplot(data= auc[:20], y= 'index', x = 0)

Jak widzimy jedynie 10 pierwszych zmiennych ma stosunkowo spore gini. Jednak już po 6 zmiennej wynik mocno spada.

In [ ]:
fcol = fcol[:6]+[' Destination Port','Source IP3','Source IP2','Source IP1']
fcol

Wzięliśmy 6 najbardziej predykcjnych zmiennych. Jak widzimy wszystkie przekazują nam informacje nie na temat samych pakietów co ich drogi wewnątrz sieci. Dorzuciliśmy Destination Port i resztę Source IP bo wydaje się, że powinny uzupełniać informacje na temat ich drogi. Może te zmienne samodzielnie są mało predykcyjne, lecz oryginalnie gdy się je połączy z resztą niosą pełną informacje na temat drogi pakietów.

### Hiperparametryzacja

In [ ]:
X_learn, X_check, Y_learn, Y_check = train_test_split(
    X_train,
    y_train,
    test_size=0.2, random_state=7)

X_learn, X_val, Y_learn, Y_val = train_test_split(
    X_learn,
    Y_learn,
    test_size=0.2, random_state=7)


Będziemy dobierać parametr po parametrze. Zaczniemy od max_depth. Gdy zacieśnimy obszar gdzie może być maximum odpalimy grid searcha.

In [ ]:
for i in range(2,20):
    model=XGBClassifier(random_state=1,
                        booster='gbtree', 
                        max_depth=i
                        )
    model.fit(X_learn[fcol], Y_learn)
    print(i,round(model.score(X_val[fcol],Y_val),3))

Różnice wewnątrz różnych wartości learning_rate i n_estimators są niewielkie, więc zrobimy wykres tylko w zależności od wysokości drzew.

In [ ]:
# Wykres tree depth accuracy !!!!!!!!
# Są stare dane bo wykres statyczny




plt.rc('axes', axisbelow=True)
plt.figure(figsize=(10,6)).clf()
sns.pointplot(x = list(range(2,20)), y = [0.524,0.542,0.56,0.584,0.617,0.648,0.674,0.695,
                                    0.708,0.724,0.736,0.739,0.746,0.746,0.749,0.749,0.75,0.75],color=".1",markers="s")
plt.ylabel('Accuracy', fontsize=16)
plt.xlabel('Tree depth', fontsize=16)
plt.title("How tree depth affect accuracy", fontsize=26)
plt.xticks(list(range(2,20,2)))
plt.grid()

Po 16 już nie ma większej różnicy,a czas obliczeń mocno wzrasta, więc wybierzemy obszar od 14 do 16. Teraz sprawdzimy n_estimators dla max_depth = 15 jako środek przedziału.

In [ ]:
for i in range(1, 7):
    model=XGBClassifier(random_state=1,
                        n_estimators = 50*i,
                        booster='gbtree', 
                        max_depth=15,
                        tree_method = 'gpu_hist'
                        )
    model.fit(X_learn[fcol], Y_learn)
    print(50*i,round(model.score(X_val[fcol],Y_val),3))

Po 200 już nie dalej wyniki nie rosną. Zatem zatrzymamy się w przedziale od 100 do 200. Sprawdzimy jeszcze learning_rate przy 100 drzewach.

In [ ]:
for i in range(1, 10):
    model=XGBClassifier(random_state=1,
                        n_estimators = 100,
                        learning_rate = i/10,
                        booster='gbtree', 
                        max_depth=15,
                        tree_method = 'gpu_hist'
                        )
    model.fit(X_learn[fcol], Y_learn)
    print(i/10,round(model.score(X_val[fcol],Y_val),4))

Tutaj wybierzemy przedział od 0.6 do 0.8.

Odpalamy grid searcha na naszych przedziałach, żeby znaleźć najlepsze parametry. Trochę zmniejszyliśmy przedział n_estimators, żeby przyśpieszyć obliczenia.

In [ ]:
opt = XGBClassifier(random_state=1,
                        booster='gbtree', 
                        tree_method = 'gpu_hist'
                        )
max_depth = list(range(14,17))
learning_rate = [i/100 for i in range(65, 76,2)]
n_estimators = list(range(100,160,20))
param_grid = dict(max_depth=max_depth,learning_rate = learning_rate,n_estimators = n_estimators)

In [ ]:
#Skleimy z powrotem learna
X_learn= pd.concat([X_learn,X_val])
Y_learn = pd.concat([Y_learn,Y_val])

In [ ]:
#grid = GridSearchCV(estimator=opt, param_grid=param_grid, cv = 3, n_jobs=-1)
#grid_result = grid.fit(X_learn[fcol],Y_learn)

In [ ]:
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Użyjemy parametrów wyliczonych przez grida i sprawdzimy czy dodatkowe kolumny o niskim auc poprawiają wyniki.

In [ ]:
model = XGBClassifier(random_state=1,
                        learning_rate= 0.67,
                        max_depth = 16,
                        n_estimators = 120,
                        booster='gbtree'
                        )
model.fit(X_learn[fcol[:6]],Y_learn) # tylko 6 najlepiej predykcyjnych kolumn
model.score(X_check[fcol[:6]],Y_check)

In [ ]:
model = XGBClassifier(random_state=1,
                        learning_rate= 0.67,
                        max_depth = 16,
                        n_estimators = 120,
                        booster='gbtree'
                        )
model.fit(X_learn[fcol],Y_learn) # wszystkie wybrane do modelu
model.score(X_check[fcol],Y_check)

Jak widać dodatkowe kolumny z niskim auc zwiększają accuracy. Wcześniejsze testy pokazały, że kolejne kolumny wybierane względem auc pogarszały wyniki. Zdajemy sobie sprawę, że hiperparametry były dostosowywane pod większą liczbę kolumn, ale pomimo to różnica jest spora między wyborem kolumn a między różnymi parametrami.

### Wstępne wyniki

In [ ]:
results=cross_val_score(model, X_learn[fcol], Y_learn)
print(np.mean(results), np.std(results))

In [ ]:
model.score(X_check[fcol],Y_check)

In [ ]:
y_predicted = model.predict(X_check[fcol])
sns.heatmap(confusion_matrix(Y_check,y_predicted),annot=True,cmap='Blues', fmt='g')

In [ ]:
plot_importance(model)
plt.show()

### Końcowe wyniki (tu jeszcze czekamy na walidacje, tak poglądowo sprawdziliśmy)

In [ ]:
model = XGBClassifier(random_state=1,
                        learning_rate= 0.67,
                        max_depth = 16,
                        n_estimators = 120,
                        booster='gbtree'
                        )
model.fit(X_train[fcol],y_train)
model.score(X_test[fcol],y_test)

In [ ]:
model.score(X_train[fcol],y_train)

In [ ]:
results=cross_val_score(model, X_train[fcol], y_train)
print(np.mean(results), np.std(results))

In [ ]:
y_predicted = model.predict(X_test[fcol])
sns.heatmap(confusion_matrix(y_test,y_predicted),annot=True,cmap='Blues', fmt='g')

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, model.predict_proba(X_test[fcol]), multi_class='ovr')

In [ ]:
# Wykres confusion_matrix !!!!!!!!



axis_labels = ['Benign', 'Scareware', 'Adware', 'SMS Malware']
conf = confusion_matrix(y_test,y_predicted)

#labels
plt.figure(figsize=(10,7)).clf()
labels = np.zeros((len(conf), len(conf)), dtype=object)
for i,row in enumerate(conf):
    for j,cell in enumerate(row):
        labels[i][j] = f"{cell}\n{round(cell*100/sum(conf[i]),1)}%" 

sns.heatmap(conf,annot=labels,
            cmap='Blues', fmt = '',  
            yticklabels=axis_labels,
            xticklabels=axis_labels)
plt.ylabel('True label', fontsize=16)
plt.yticks(rotation=0)
plt.xlabel('Predicted label', fontsize=16)
plt.title("Confusion matrix", fontsize=26)
plt.legend('', frameon=False)

In [ ]:
# Wykres roc_curve !!!!!!!!



plt.figure(figsize=(10,10)).clf()

# plt.subplot(1, 2, 1)
axis_labels = ['Benign', 'Scareware', 'Adware', 'SMS Malware']
y = pd.get_dummies(y_test)
y_proba = model.predict_proba(X_test[fcol])

for i in range(4):
    auc = roc_auc_score(y.iloc[:,i],y_proba[:, i])
    fpr, tpr, _ = roc_curve(y.iloc[:,i],y_proba[:, i])
    plt.plot(fpr,tpr,label=f"AUC for {axis_labels[i]}={'{:0.3f}'.format(auc)}", linewidth = 3)


plt.plot([0, 1], [0, 1],color = 'black', linestyle='--')
plt.legend(loc=4,fontsize='x-large')
plt.title("AUC ROC curve for each class vs rest", fontsize=26)
plt.ylabel("True Positive rate", fontsize=16)
plt.xlabel("False Positive rate", fontsize=16)
plt.grid()


# plt.subplot(1, 2, 2)

# auc =roc_auc_score(y_test, model.predict_proba(X_test[fcol]), multi_class='ovr')
# # fpr, tpr, _ = roc_curve(y.iloc,y_proba)
# RocCurveDisplay.from_predictions(y_test, model.predict_proba(X_test[fcol]))
# # plt.plot(fpr,tpr,label=f"AUC ={'{:0.3f}'.format(auc)}", linewidth = 3)
    
    
    
# plt.plot([0, 1], [0, 1],color = 'black', linestyle='--')
# plt.title("AUC ROC curve for model", fontsize=26)
# plt.ylabel("True Positive rate", fontsize=16)
# plt.xlabel("False Positive rate", fontsize=16)
# plt.grid()

# plt.show

In [ ]:
# Wykres Feature importance !!!!!!!!


plt.rcParams['figure.figsize'] = [10, 8]
plot_importance(model,show_values=False, color="gray", height=0.6,edgecolor='black')

plt.title("Feature importance", fontsize=26)
plt.ylabel("Feature name", fontsize=16)
plt.xlabel("F score", fontsize=16)
plt.xticks([0,20_000,40_000,60_000,80_000,100_000,120_000], ["0","20K","40K","60K","80K","100K","120K"])
plt.show()